In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt
from scipy.special import softmax

In [ ]:
rewards = np.zeros(16)
rewards[3] = 10
rewards[2] = -1
rewards[11] = -1
rewards[10] = -1

terminal_state = 3
state_action_logprobs = np.random.random((16,4))
alpha = 0.005

score_log = []

In [ ]:
plt.imshow(rewards.reshape(4, 4))

In [ ]:
state_transition_table = np.genfromtxt("state_transitions.csv", delimiter=",").astype(int)

In [ ]:
def returns_calc(rewards):
    returns = []
    R = 0
    gamma = 0.9
    for i in reversed(range(len(rewards))):
        R = rewards[i] + gamma * R
        returns.insert(0, R)
    return returns

In [ ]:
def test_agent():
    state = 12
    done = False
    steps = 0
    total_rewards = 0
    states_log = []
    while (not(state == terminal_state)) and steps<30:
        states_log.append(state)
        action = np.argmax(np.random.multinomial(1, softmax(state_action_logprobs[state]), size=1))
        state = state_transition_table[state, action]
        total_rewards += rewards[state]
        steps += 1
    states_log.append(state)
    return total_rewards, states_log

In [ ]:
for _ in range(1000):
    state = 12
    state_log = []
    reward_log = []
    action_log = []
    values_log = []

    steps = 0

    while (not(state == terminal_state)) and steps<30:
        action = np.argmax(np.random.multinomial(1, softmax(state_action_logprobs[state]), size=1))
        
        state_log.append(state)
        action_log.append(action)

        state = state_transition_table[state, action]
        reward_log.append(rewards[state])

        steps += 1
            
    state_returns = returns_calc(reward_log)
    advantage = np.array(state_returns)
    
    state_action_logprobs[state_log, action_log] = state_action_logprobs[state_log, action_log] + alpha*advantage
    
    score_log.append(test_agent()[0])

In [ ]:
fig, ax1 = plt.subplots(1)
fig.set_size_inches(18.5, 10.5)
ax1.imshow(softmax(state_action_logprobs, 1).reshape(16, 4))

for (j,i), label in np.ndenumerate(softmax(state_action_logprobs, 1).reshape(16, 4).round(1)):
    ax1.text(i,j,label,ha='center',va='center')

In [ ]:
plt.plot(score_log)

In [ ]:
_, state_log = test_agent()
state_view = np.zeros(16)
state_view[state_log] = 1
plt.imshow(state_view.reshape(4,4))